In [18]:
import pandas as pd
import numpy as np
# import xgboost as xgb
from tqdm import tqdm
# from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Embedding, GlobalAveragePooling1D, Conv1D
from keras.utils import np_utils, to_categorical
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D, AveragePooling1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from tensorflow.keras.optimizers import Adam

# embeddig_dims, dropout, pooling종류, maxlen 조정

In [34]:
train = pd.read_csv('../raw/train.csv')
test = pd.read_csv('../raw/test_x.csv')
sub = pd.read_csv('../raw/sample_submission.csv')

In [35]:
def preprocess(text):
    text = text.replace("' ", " ' ")
    signs = set(',.:;?!"') 
    prods = set(text) & signs
    if not prods:
        return text

    for sign in prods:
        text = text.replace(sign, ' {} '.format(sign) )
    return text


def create_docs(train, n_gram_max=2):
    def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('—'.join(q[w_index:w_index+n]))
            return q + ngrams
        
    docs = []
    for doc in train.text:
        doc = preprocess(doc).split()
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
    
    return docs

min_count = 2

docs = create_docs(train)
tokenizer = Tokenizer(lower=True, filters='')
tokenizer.fit_on_texts(docs)
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

tokenizer = Tokenizer(num_words=num_words, lower=True, filters='')
tokenizer.fit_on_texts(docs)
docs = tokenizer.texts_to_sequences(docs)

maxlen = 256

docs = pad_sequences(sequences=docs, maxlen=maxlen)


input_dim = np.max(docs) + 1
embedding_dims = 20

In [36]:
docs2 = create_docs(test)
docs2 = tokenizer.texts_to_sequences(docs2)
docs2 = pad_sequences(sequences=docs2, maxlen=maxlen)

In [37]:
y = train['author']
y = to_categorical(y)

4376

In [43]:
es = EarlyStopping(monitor='val_loss', patience=20, verbose=1)
mc=ModelCheckpoint("./fasttext.h5",save_best_only=True,verbose=1)
reLR = ReduceLROnPlateau(patience=13,verbose=1,factor=0.65)

mskf = MultilabelStratifiedKFold(n_splits=3, shuffle=True, random_state=0)

predict=0
loop = 0
for train_index, valid_index in mskf.split(docs,y) :
    loop +=1
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~{}번째~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'.format(loop))
    x_train=docs[train_index]
    x_valid=docs[valid_index]
    y_train=y[train_index]
    y_valid=y[valid_index]
    mc=ModelCheckpoint("./final_writer{}.h5".format(loop),save_best_only=True,verbose=1)

    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(Dropout(0.5))

    model.add(AveragePooling1D(pool_size=2, strides=None, padding="valid", data_format="channels_last"))
    model.add(Dropout(0.3))

    model.add(AveragePooling1D(pool_size=2, strides=None, padding="valid", data_format="channels_last"))
    model.add(Dropout(0.3))

    model.add(GlobalAveragePooling1D())
    model.add(Dropout(0.5))

    model.add(Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.001), metrics=['accuracy'])
    model.summary()
    history=model.fit(x_train, y_train, batch_size=512, validation_data=(x_valid, y_valid), epochs=10000, callbacks=[es, mc, reLR])
    predict+=model.predict(docs2)/3

C:\anaconda\envs\college4\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1번째~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, None, 20)          4084020   
_________________________________________________________________
dropout_57 (Dropout)         (None, None, 20)          0         
_________________________________________________________________
average_pooling1d_27 (Averag (None, None, 20)          0         
_________________________________________________________________
dropout_58 (Dropout)         (None, None, 20)          0         
_________________________________________________________________
average_pooling1d_28 (Averag (None, None, 20)          0         
_________________________________________________________________
dropout_59 (Dropout)         (None, None, 20)          0     

Epoch 22/10000
71/72 [============================>.] - ETA: 0s - loss: 1.1218 - accuracy: 0.6156
Epoch 00022: val_loss improved from 1.14145 to 1.12046, saving model to .\final_writer1.h5
72/72 [==============================] - 5s 63ms/step - loss: 1.1217 - accuracy: 0.6157 - val_loss: 1.1205 - val_accuracy: 0.6396
Epoch 23/10000
71/72 [============================>.] - ETA: 0s - loss: 1.1016 - accuracy: 0.6270
Epoch 00023: val_loss improved from 1.12046 to 1.10069, saving model to .\final_writer1.h5
72/72 [==============================] - 4s 62ms/step - loss: 1.1014 - accuracy: 0.6271 - val_loss: 1.1007 - val_accuracy: 0.6482
Epoch 24/10000
71/72 [============================>.] - ETA: 0s - loss: 1.0830 - accuracy: 0.6322 ETA: 
Epoch 00024: val_loss improved from 1.10069 to 1.08176, saving model to .\final_writer1.h5
72/72 [==============================] - 5s 63ms/step - loss: 1.0828 - accuracy: 0.6322 - val_loss: 1.0818 - val_accuracy: 0.6459
Epoch 25/10000
71/72 [===============

Epoch 48/10000
71/72 [============================>.] - ETA: 0s - loss: 0.7767 - accuracy: 0.7436
Epoch 00048: val_loss improved from 0.80566 to 0.79811, saving model to .\final_writer1.h5
72/72 [==============================] - 5s 64ms/step - loss: 0.7767 - accuracy: 0.7435 - val_loss: 0.7981 - val_accuracy: 0.7502
Epoch 49/10000
71/72 [============================>.] - ETA: 0s - loss: 0.7682 - accuracy: 0.7510
Epoch 00049: val_loss improved from 0.79811 to 0.79057, saving model to .\final_writer1.h5
72/72 [==============================] - 5s 65ms/step - loss: 0.7684 - accuracy: 0.7509 - val_loss: 0.7906 - val_accuracy: 0.7525
Epoch 50/10000
71/72 [============================>.] - ETA: 0s - loss: 0.7581 - accuracy: 0.7539
Epoch 00050: val_loss improved from 0.79057 to 0.78324, saving model to .\final_writer1.h5
72/72 [==============================] - 5s 64ms/step - loss: 0.7578 - accuracy: 0.7540 - val_loss: 0.7832 - val_accuracy: 0.7529
Epoch 51/10000
71/72 [=====================

Epoch 74/10000
71/72 [============================>.] - ETA: 0s - loss: 0.5888 - accuracy: 0.8164
Epoch 00074: val_loss improved from 0.65506 to 0.65096, saving model to .\final_writer1.h5
72/72 [==============================] - 5s 65ms/step - loss: 0.5888 - accuracy: 0.8166 - val_loss: 0.6510 - val_accuracy: 0.7965
Epoch 75/10000
71/72 [============================>.] - ETA: 0s - loss: 0.5815 - accuracy: 0.8188
Epoch 00075: val_loss improved from 0.65096 to 0.64676, saving model to .\final_writer1.h5
72/72 [==============================] - 5s 65ms/step - loss: 0.5816 - accuracy: 0.8188 - val_loss: 0.6468 - val_accuracy: 0.8024
Epoch 76/10000
71/72 [============================>.] - ETA: 0s - loss: 0.5780 - accuracy: 0.8194
Epoch 00076: val_loss improved from 0.64676 to 0.64256, saving model to .\final_writer1.h5
72/72 [==============================] - 5s 64ms/step - loss: 0.5783 - accuracy: 0.8192 - val_loss: 0.6426 - val_accuracy: 0.7974
Epoch 77/10000
71/72 [=====================

...

Epoch 84/10000
71/72 [============================>.] - ETA: 0s - loss: 0.5193 - accuracy: 0.8413
Epoch 00084: val_loss improved from 0.61220 to 0.60819, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 64ms/step - loss: 0.5193 - accuracy: 0.8413 - val_loss: 0.6082 - val_accuracy: 0.8078
Epoch 85/10000
71/72 [============================>.] - ETA: 0s - loss: 0.5124 - accuracy: 0.8446 ETA: 0s - loss: 0.5124 - accuracy: 0.
Epoch 00085: val_loss improved from 0.60819 to 0.60540, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 64ms/step - loss: 0.5128 - accuracy: 0.8446 - val_loss: 0.6054 - val_accuracy: 0.8075
Epoch 86/10000
71/72 [============================>.] - ETA: 0s - loss: 0.5086 - accuracy: 0.8445
Epoch 00086: val_loss improved from 0.60540 to 0.60139, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 64ms/step - loss: 0.5086 - accuracy: 0.8446 - val_loss: 0.6014 - val_accuracy: 0.8089
Epoch

Epoch 110/10000
71/72 [============================>.] - ETA: 0s - loss: 0.4100 - accuracy: 0.8777
Epoch 00110: val_loss improved from 0.54240 to 0.54044, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 65ms/step - loss: 0.4099 - accuracy: 0.8778 - val_loss: 0.5404 - val_accuracy: 0.8246
Epoch 111/10000
71/72 [============================>.] - ETA: 0s - loss: 0.4044 - accuracy: 0.8802
Epoch 00111: val_loss improved from 0.54044 to 0.53888, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 64ms/step - loss: 0.4046 - accuracy: 0.8801 - val_loss: 0.5389 - val_accuracy: 0.8250
Epoch 112/10000
71/72 [============================>.] - ETA: 0s - loss: 0.4034 - accuracy: 0.8804
Epoch 00112: val_loss improved from 0.53888 to 0.53654, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 63ms/step - loss: 0.4037 - accuracy: 0.8803 - val_loss: 0.5365 - val_accuracy: 0.8242
Epoch 113/10000
71/72 [=================

Epoch 136/10000
71/72 [============================>.] - ETA: 0s - loss: 0.3288 - accuracy: 0.9055
Epoch 00136: val_loss improved from 0.50095 to 0.50006, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 63ms/step - loss: 0.3286 - accuracy: 0.9055 - val_loss: 0.5001 - val_accuracy: 0.8312
Epoch 137/10000
71/72 [============================>.] - ETA: 0s - loss: 0.3273 - accuracy: 0.9039
Epoch 00137: val_loss improved from 0.50006 to 0.49873, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 63ms/step - loss: 0.3273 - accuracy: 0.9039 - val_loss: 0.4987 - val_accuracy: 0.8337
Epoch 138/10000
71/72 [============================>.] - ETA: 0s - loss: 0.3276 - accuracy: 0.9050 ETA: 0s - loss: 0.3270 - accura
Epoch 00138: val_loss did not improve from 0.49873
72/72 [==============================] - 5s 64ms/step - loss: 0.3280 - accuracy: 0.9050 - val_loss: 0.4994 - val_accuracy: 0.8323
Epoch 139/10000
71/72 [=========================

Epoch 162/10000
71/72 [============================>.] - ETA: 0s - loss: 0.2725 - accuracy: 0.9228
Epoch 00162: val_loss improved from 0.47771 to 0.47639, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 65ms/step - loss: 0.2728 - accuracy: 0.9228 - val_loss: 0.4764 - val_accuracy: 0.8386
Epoch 163/10000
71/72 [============================>.] - ETA: 0s - loss: 0.2731 - accuracy: 0.9207
Epoch 00163: val_loss improved from 0.47639 to 0.47603, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 63ms/step - loss: 0.2728 - accuracy: 0.9210 - val_loss: 0.4760 - val_accuracy: 0.8382
Epoch 164/10000
71/72 [============================>.] - ETA: 0s - loss: 0.2751 - accuracy: 0.9202
Epoch 00164: val_loss improved from 0.47603 to 0.47508, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 65ms/step - loss: 0.2751 - accuracy: 0.9202 - val_loss: 0.4751 - val_accuracy: 0.8388
Epoch 165/10000
71/72 [=================

Epoch 189/10000
71/72 [============================>.] - ETA: 0s - loss: 0.2328 - accuracy: 0.9325
Epoch 00189: val_loss improved from 0.46482 to 0.46410, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 64ms/step - loss: 0.2328 - accuracy: 0.9325 - val_loss: 0.4641 - val_accuracy: 0.8403
Epoch 190/10000
71/72 [============================>.] - ETA: 0s - loss: 0.2306 - accuracy: 0.9335
Epoch 00190: val_loss did not improve from 0.46410
72/72 [==============================] - 4s 62ms/step - loss: 0.2306 - accuracy: 0.9334 - val_loss: 0.4645 - val_accuracy: 0.8397
Epoch 191/10000
71/72 [============================>.] - ETA: 0s - loss: 0.2269 - accuracy: 0.9359
Epoch 00191: val_loss improved from 0.46410 to 0.46384, saving model to .\final_writer3.h5
72/72 [==============================] - 5s 63ms/step - loss: 0.2269 - accuracy: 0.9359 - val_loss: 0.4638 - val_accuracy: 0.8396
Epoch 192/10000
71/72 [============================>.] - ETA: 0s - loss: 0.2287 

In [44]:
# submit
sub=sub.set_index('index')
sub[sub.columns] = predict
sub=sub.reset_index()
sub.to_csv('../predict/last_sub.csv', index=False)

In [45]:
sub

,index,0,1,2,3,4
0,0,9.849198e-04,9.310905e-01,6.745458e-02,3.890575e-04,8.097703e-05
1,1,7.906828e-04,9.939961e-01,1.922075e-04,2.902060e-03,2.118804e-03
2,2,9.999683e-01,2.084729e-05,8.651201e-07,2.745212e-06,7.190980e-06
3,3,6.763933e-04,5.854318e-09,9.978395e-01,1.604136e-09,1.484202e-03
4,4,9.959507e-01,3.394811e-03,1.237831e-04,2.220520e-04,3.086511e-04
...,...,...,...,...,...,...
19612,19612,1.989385e-11,1.000000e+00,9.355009e-15,5.344671e-16,1.850151e-14
19613,19613,4.380386e-06,1.646134e-07,8.965841e-07,2.409772e-07,9.999943e-01
19614,19614,3.235716e-03,9.966842e-01,1.232165e-06,6.965635e-05,9.160052e-06
19615,19615,9.275119e-07,9.999962e-01,1.297929e-06,1.105815e-06,4.999864e-07
